In [1]:
import gzip
import math
import numpy
import random
import sklearn
import string
from collections import defaultdict
from nltk.stem.porter import *
from sklearn import linear_model
from gensim.models import Word2Vec
import dateutil
from scipy.sparse import lil_matrix # To build sparse feature matrices, if you like
import pandas as pd

import html
import random
import json

In [2]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

In [3]:
# allRatings = [l for l in parse("ratebeer.json.gz")] # sampled has 300,000 reviews
ratings = [l for l in parse("sampled_ratebeer.json.gz")][0] # sampled has 300,000 reviews

# Sample and compress dataset (ALREADY RAN DONT RUN)

In [ ]:
# # Random sample the dataset because it's too big

# sampleSize = 300000
# sampledAllRatings = random.sample(ratings, sampleSize)

In [ ]:
# import json

# with open("sampled_ratebeer.json", "w") as f:
#     json.dump(sampledAllRatings, f)

In [ ]:
# import gzip
# import shutil

# # File paths
# input_file = 'sampled_ratebeer.json'      # Your existing JSON file
# output_file = 'sampled_ratebeer.json.gz'  # Desired compressed file

# # Compress the JSON file
# with open(input_file, 'rb') as f_in:
#     with gzip.open(output_file, 'wb') as f_out:
#         shutil.copyfileobj(f_in, f_out)

# print(f"Compressed {input_file} to {output_file}")


Compressed sampled_ratebeer.json to sampled_ratebeer.json.gz


# Clean data

Check missing labels (style)

In [4]:
for rating in ratings:
    if 'beer/style' in rating:
        rating['beer/style'] = html.unescape(rating['beer/style'])

In [5]:
styles = set()
no_style = []

for rating in ratings:
    if 'beer/style' in rating:
        styles.add(rating['beer/style'])
    else:
        no_style.append(rating)

len(styles), len(no_style)

(89, 0)

In [15]:
allKeys = set(ratings[0].keys())

NameError: name 'ratings' is not defined

Check missing features

In [7]:
hasMissingData = []

for rating in ratings:
    if allKeys != set(rating.keys()):
        hasMissingData.append(rating)

In [8]:
hasMissingData

[]